In [ ]:
import wandb
import pandas as pd

# WandB 프로젝트 불러오기
api = wandb.Api()
runs = api.runs("document-classification")

# 각 지표를 담을 리스트 초기화
run_data = []

for run in runs:
    # 각 run에서 필요한 지표를 추출
    run_data.append({
        "run_name": run.name,
        "train_loss": run.summary.get("final_train_loss"),
        "train_accuracy": run.summary.get("final_train_accuracy"),
        "train_f1": run.summary.get("final_train_f1"),
        "val_loss": run.summary.get("final_valid_loss"),
        "val_accuracy": run.summary.get("final_valid_accuracy"),
        "val_f1": run.summary.get("final_valid_f1"),
        "test_loss": run.summary.get("final_test_loss"),
        "test_accuracy": run.summary.get("final_test_accuracy"),
        "test_f1": run.summary.get("final_test_f1")
    })

# DataFrame으로 변환
df = pd.DataFrame(run_data)

# 각 지표의 중요도 설정 (가중치)
# 여기서는 단순히 모든 지표를 동일하게 고려
weights = {
    "train_loss": -1,
    "train_accuracy": 1,
    "train_f1": 1,
    "val_loss": -1,
    "val_accuracy": 1,
    "val_f1": 1,
    "test_loss": -1,
    "test_accuracy": 1,
    "test_f1": 1
}

# 가중치를 이용하여 종합 점수 계산
df["combined_score"] = (
    weights["train_loss"] * df["train_loss"] +
    weights["train_accuracy"] * df["train_accuracy"] +
    weights["train_f1"] * df["train_f1"] +
    weights["val_loss"] * df["val_loss"] +
    weights["val_accuracy"] * df["val_accuracy"] +
    weights["val_f1"] * df["val_f1"] +
    weights["test_loss"] * df["test_loss"] +
    weights["test_accuracy"] * df["test_accuracy"] +
    weights["test_f1"] * df["test_f1"]
)

# 종합 점수 기준 정렬
sorted_df = df.sort_values(by="combined_score", ascending=False)

# 상위 모델 선택
top_models = sorted_df.head(5)

# 결과 출력
print(top_models)

In [ ]:
import wandb
import pandas as pd
import numpy as np

# WandB 프로젝트 불러오기
api = wandb.Api()
runs = api.runs("document-classification")

# 각 지표를 담을 리스트 초기화
run_data = []

for run in runs:
    # 각 run에서 필요한 지표를 추출
    run_data.append({
        "run_name": run.name,
        "train_loss": run.summary.get("final_train_loss"),
        "train_accuracy": run.summary.get("final_train_accuracy"),
        "train_f1": run.summary.get("final_train_f1"),
        "val_loss": run.summary.get("final_valid_loss"),
        "val_accuracy": run.summary.get("final_valid_accuracy"),
        "val_f1": run.summary.get("final_valid_f1"),
        "test_loss": run.summary.get("final_test_loss"),
        "test_accuracy": run.summary.get("final_test_accuracy"),
        "test_f1": run.summary.get("final_test_f1")
    })

# DataFrame으로 변환
df = pd.DataFrame(run_data)

def calculate_updated_score(row):
    # 평가 세트(valid + test) 성능 계산
    eval_f1 = (row['val_f1'] + row['test_f1']) / 2
    eval_accuracy = (row['val_accuracy'] + row['test_accuracy']) / 2
    eval_loss = (row['val_loss'] + row['test_loss']) / 2

    # 성능 점수 계산
    performance_score = 0.5 * eval_f1 + 0.3 * eval_accuracy - 0.2 * eval_loss

    # 과적합 점수 계산
    overfitting_score = (row['train_accuracy'] - eval_accuracy) + (row['train_f1'] - eval_f1)
    overfitting_penalty = 1 / (1 + np.exp(-5 * (overfitting_score - 0.1)))  # 시그모이드 함수

    # 일관성 점수 계산 (valid와 test 간의 차이, 작을수록 좋음)
    consistency_score = 1 / (1 + abs(row['val_f1'] - row['test_f1']) + abs(row['val_accuracy'] - row['test_accuracy']))

    # 최종 점수 계산
    final_score = performance_score * (1 - 0.3 * overfitting_penalty) * consistency_score

    return final_score

# 종합 점수 계산 및 정렬
df['combined_score'] = df.apply(calculate_updated_score, axis=1)
sorted_df = df.sort_values(by="combined_score", ascending=False)

# 상위 모델 선택
top_models = sorted_df.head(5)

# 결과 출력
print(top_models)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
from sklearn.metrics import f1_score
import matplotlib.font_manager as fm
from matplotlib.font_manager import FontProperties

# 클래스 이름 정의
class_names = {
    0: "계좌번호(손글씨)", 1: "임신출산 진료비 지급 신청서", 2: "자동차 계기판", 3: "입퇴원 확인서", 4: "진단서", 
    5: "운전면허증", 6: "진료비영수증", 7: "통원/진료 확인서", 8: "주민등록증", 9: "여권", 
    10: "진료비 납입 확인서", 11: "약제비 영수증", 12: "처방전", 13: "이력서", 14: "소견서", 
    15: "자동차 등록증", 16: "자동차 번호판"
}

def load_predictions(file_paths):
    """여러 모델의 예측 결과를 로드합니다."""
    predictions = {}
    for path in file_paths:
        model_name = os.path.basename(path).split('.')[0]
        df = pd.read_csv(path)
        # ID 컬럼 중복 제거 및 예측 컬럼 이름 변경
        df = df[['ID', 'target']].rename(columns={'target': model_name})
        predictions[model_name] = df
    return predictions

def find_different_predictions(predictions):
    """모든 모델에서 다르게 예측한 항목을 찾습니다."""
    # 모든 예측을 하나의 DataFrame으로 병합
    all_predictions = predictions[list(predictions.keys())[0]]
    for model, df in list(predictions.items())[1:]:
        all_predictions = pd.merge(all_predictions, df, on='ID', suffixes=('', f'_{model}'))
    
    # 예측 컬럼만 선택
    prediction_columns = [col for col in all_predictions.columns if col != 'ID']
    
    # 예측이 다른 행만 선택
    different_predictions = all_predictions[all_predictions[prediction_columns].nunique(axis=1) > 1]
    return different_predictions

def calculate_macro_f1(ground_truth, predictions):
    """Macro F1 점수를 계산합니다."""
    return f1_score(ground_truth, predictions, average='macro')

def plot_error_distribution(ground_truth, predictions):
    """각 클래스별 오류 예측 개수를 bar plot으로 표시합니다."""
    error_counts = {i: 0 for i in range(len(class_names))}  # 모든 클래스에 대해 초기화
    for gt, pred in zip(ground_truth, predictions):
        if gt != pred:
            error_counts[gt] += 1
    
    classes = sorted(error_counts.keys())
    counts = [error_counts[c] for c in classes]
    class_labels = [f"{c}\n{class_names[c]}" for c in classes]
    
    # 나눔 폰트 설정
    font_path = './font/NanumGothic.otf'
    font_prop = FontProperties(fname=font_path)
    
    plt.figure(figsize=(20, 10))
    bars = plt.bar(class_labels, counts)
    plt.title("오류 예측 개수 (클래스별)", fontproperties=font_prop, fontsize=16)
    plt.xlabel("클래스", fontproperties=font_prop, fontsize=14)
    plt.ylabel("오류 개수", fontproperties=font_prop, fontsize=14)
    plt.xticks(rotation=45, ha='center')
    
    # x축 레이블에 폰트 적용
    ax = plt.gca()
    ax.set_xticklabels(class_labels, fontproperties=font_prop, fontsize=10)
    
    plt.tight_layout()
    
    # 각 막대 위에 값 표시
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height}',
                 ha='center', va='bottom', fontproperties=font_prop)
    
    plt.show()  

def display_images_and_predictions(different_predictions, image_dir, ground_truth_name, prediction_name, class_filter=None, max_images_per_class=3):
    """다르게 예측된 항목의 이미지와 예측 결과를 표시합니다."""
    # 나눔 폰트 설정
    font_path = './font/NanumGothic.otf'
    font_prop = FontProperties(fname=font_path)
    
    # 클래스별로 이미지 분류
    class_images = {c: [] for c in range(len(class_names))}
    for idx, row in different_predictions.iterrows():
        ground_truth = row[ground_truth_name]
        if class_filter is None or ground_truth in class_filter:
            class_images[ground_truth].append(row)
    
    # 표시할 이미지 선택
    images_to_display = []
    for c, imgs in class_images.items():
        if class_filter is None or c in class_filter:
            images_to_display.extend(imgs[:max_images_per_class])
    
    # 이미지 표시
    num_images = len(images_to_display)
    rows = (num_images - 1) // 4 + 1
    fig, axs = plt.subplots(rows, 4, figsize=(20, 5*rows))
    
    for i, row in enumerate(images_to_display):
        ax = axs[i//4, i%4] if rows > 1 else axs[i%4]
        
        image_id = row['ID']
        ground_truth = row[ground_truth_name]
        prediction = row[prediction_name]
        image_path = os.path.join(image_dir, image_id)
        
        if os.path.exists(image_path):
            img = Image.open(image_path)
            img = img.resize((300, 300), Image.LANCZOS)
            ax.imshow(img)
            ax.axis('off')
            id = f"ID: {image_id}"
            comparison_text = f"정답: {ground_truth} ({class_names[ground_truth]})\n예측: {prediction} ({class_names[prediction]})"
            ax.set_title(f"{id}\n{comparison_text}", fontproperties=font_prop, fontsize=10)
        else:
            ax.text(0.5, 0.5, f"Image not found\nfor ID: {image_id}", ha='center', va='center')
    
    # 빈 서브플롯 제거
    for i in range(num_images, rows*4):
        ax = axs[i//4, i%4] if rows > 1 else axs[i%4]
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"총 {num_images}개의 이미지가 표시되었습니다.")

# # 예측 파일 경로
# file_paths = [
#     "output/9609.csv",
#     "output/20240808-194900_final.pt.csv",
# ]

# # 이미지 디렉토리 경로
# image_dir = "data/test"

# # 예측 로드
# predictions = load_predictions(file_paths)

# # 다른 예측 찾기
# different_predictions = find_different_predictions(predictions)

# # # 결과 출력
# # print(different_predictions)

# # Macro F1 점수 계산
# ground_truth_name = os.path.basename(file_paths[0]).split('.')[0]
# prediction_name = os.path.basename(file_paths[-1]).split('.')[0]
# ground_truth = predictions[ground_truth_name][ground_truth_name]
# prediction = predictions[prediction_name][prediction_name]
# macro_f1 = calculate_macro_f1(ground_truth, prediction)
# print(f"Macro F1 Score: {macro_f1}")

# # 오류 분포 그래프 표시
# plot_error_distribution(ground_truth, prediction)

# # 모든 클래스에 대해 클래스당 1개씩 이미지 표시
# # display_images_and_predictions(different_predictions, image_dir, ground_truth_name, prediction_name, class_filter=None, max_images_per_class=2)

# # 특정 클래스(예: 0, 1, 2)에 대해 클래스당 최대 3개씩 이미지 표시
# display_images_and_predictions(different_predictions, image_dir, ground_truth_name, prediction_name, class_filter=[3, 7, 14], max_images_per_class=3)